<a href="https://colab.research.google.com/github/KhamessiTaha/ALPR/blob/main/ALPR_License_Plate_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install opencv-python-headless pytesseract
!apt-get install -y tesseract-ocr
!apt-get install -y libgl1-mesa-glx

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (11.1 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123576 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e

In [6]:
import cv2
import numpy as np
import pytesseract
from google.colab.patches import cv2_imshow

weights_path = "/content/drive/My Drive/ALPR/yolov4-obj_best.weights"
cfg_path = "/content/drive/My Drive/ALPR/yolov4-obj.cfg"
names_path = "/content/drive/My Drive/ALPR/custom.names"
video_path = "/content/drive/My Drive/ALPR/sample.mp4"
output_video_path = "/content/drive/My Drive/ALPR/output_video.mp4"


net = cv2.dnn.readNet(weights_path, cfg_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]


with open(names_path, "r") as f:
    classes = [line.strip() for line in f.readlines()]


pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'


In [12]:

cap = cv2.VideoCapture(video_path)


if not cap.isOpened():
    print("Error: Could not open video.")
else:

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    print(f"Frame Width: {frame_width}, Frame Height: {frame_height}, FPS: {fps}")


    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))


    if not out_writer.isOpened():
        print("Error: Could not create video writer.")
    else:
        print("Video writer successfully created.")
        print(f"Video writer type: {type(out_writer)}")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            height, width, channels = frame.shape


            blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
            net.setInput(blob)
            outs = net.forward(output_layers)


            class_ids = []
            confidences = []
            boxes = []

            for out_detection in outs:
                for detection in out_detection:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5:  # confidence threshold
                        center_x = int(detection[0] * width)
                        center_y = int(detection[1] * height)
                        w = int(detection[2] * width)
                        h = int(detection[3] * height)
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)


            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

            for i in range(len(boxes)):
                if i in indexes:
                    x, y, w, h = boxes[i]
                    label = str(classes[class_ids[i]])
                    confidence = confidences[i]


                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


                    if x >= 0 and y >= 0 and x+w <= width and y+h <= height:

                        license_plate = frame[y:y+h, x:x+w]
                        gray_license_plate = cv2.cvtColor(license_plate, cv2.COLOR_BGR2GRAY)
                        text = pytesseract.image_to_string(gray_license_plate, config='--psm 8')


                        cv2.putText(frame, text.strip(), (x, y + h + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


            out_writer.write(frame)

        cap.release()
        out_writer.release()
        print("Video processing complete and saved to Google Drive.")


Frame Width: 1920, Frame Height: 1080, FPS: 30.0
Video writer successfully created.
Video writer type: <class 'cv2.VideoWriter'>
Video processing complete and saved to Google Drive.
